<a href="https://colab.research.google.com/github/cristian-rincon/escuela-datascience/blob/master/introduccion_web_scraping/project_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping página El Tiempo

> eltiempo.com

In [1]:
import requests

In [2]:
url = 'https://eltiempo.com'

In [3]:
eltiempo = requests.get(url)

In [4]:
eltiempo.status_code

200

In [5]:
# eltiempo.text

In [6]:
# eltiempo.content

In [36]:
# eltiempo.headers

In [8]:
eltiempo.request.headers

{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [9]:
eltiempo.request.method

'GET'

In [10]:
eltiempo.request.url

'https://www.eltiempo.com/'

## Identificar la información de interés

In [11]:
from bs4 import BeautifulSoup

In [12]:
soup = BeautifulSoup(eltiempo.text, 'lxml')

In [13]:
type(soup)

bs4.BeautifulSoup

In [35]:
# print(soup.prettify)

### Menú principal

In [37]:
menu = soup.find('ul', attrs={'class':'default-menu'}).find_all('li')
#menu

In [16]:
item_menu = menu[0]
item_menu

<li class="opinion first" itemprop="name">
<a href="/opinion" itemprop="url">Opinión</a>
</li>

In [17]:
# item_menu.a
item_menu.a.get_text()

'Opinión'

In [18]:
item_menu.a.get('href')

'/opinion'

In [38]:
links_menu = [item_menu.a.get('href') for item_menu in menu]
# links_menu

In [160]:
section = requests.get( url + links_menu[0])
section

<Response [200]>

In [161]:
soup_section = BeautifulSoup(section.text, 'lxml')
# soup_section.prettify()

In [162]:
feature_article = soup_section.find('div', attrs={'class':'opinion-editorial-board'})
#feature_article

In [163]:
feature_article.a.get('href')

'/opinion/editorial'

In [164]:
article_list = soup_section.find('div', attrs={'class':'row-author-articles'})
article_list.find_all('a')

[<a class="image page-link " href="/opinion/columnistas/tomas-y-jeronimo-uribe-moreno/preso-politico-columna-de-tomas-y-jeronimo-uribe-moreno-532298" id="m6683-6682-6684" itemprop="image" itemscope="" itemtype="https://schema.org/ImageObject"><meta content="80" itemprop="width"/>
 <meta content="80" itemprop="height"/>
 <meta content="https://www.eltiempo.com/files/image_80_80/uploads/2020/08/22/5f41d801a462f.jpeg" itemprop="url"/></a>,
 <a class="author-name page-link " href="/opinion/columnistas/tomas-y-jeronimo-uribe-moreno" id="m6690-1-6691">Tomás y Jerónimo Uribe Moreno</a>,
 <a class="title page-link " href="/opinion/columnistas/tomas-y-jeronimo-uribe-moreno/preso-politico-columna-de-tomas-y-jeronimo-uribe-moreno-532298" id="m6696-2-6697">Preso político</a>,
 <a class="epigraph page-link " href="/opinion/columnistas/tomas-y-jeronimo-uribe-moreno/preso-politico-columna-de-tomas-y-jeronimo-uribe-moreno-532298" id="m6702-3-6703">Álvaro Uribe es una nueva víctima de esta realidad que

### TODO - Escribir una función que reciba un objeto de bs y retorne el artículo principal y los secundarios

In [165]:
def get_articles(BSObject, f_articles_attrs='opinion-editorial-board', a_list_attrs='row-author-articles'):

    # Getting promotional article
    posts_list = []
    feature_article = BSObject.find('div', attrs={'class':f_articles_attrs})
    if feature_article:
        posts_list.append(feature_article.a.get('href'))

    # Getting list of articles
    articles_list = BSObject.find('div', attrs={'class':a_list_attrs})
    for article in article_list.find_all('a'):
        posts_list.append(article.get('href'))

    return posts_list

In [166]:
articulos = get_articles(soup_section)
articulos

['/opinion/editorial',
 '/opinion/columnistas/tomas-y-jeronimo-uribe-moreno/preso-politico-columna-de-tomas-y-jeronimo-uribe-moreno-532298',
 '/opinion/columnistas/tomas-y-jeronimo-uribe-moreno',
 '/opinion/columnistas/tomas-y-jeronimo-uribe-moreno/preso-politico-columna-de-tomas-y-jeronimo-uribe-moreno-532298',
 '/opinion/columnistas/tomas-y-jeronimo-uribe-moreno/preso-politico-columna-de-tomas-y-jeronimo-uribe-moreno-532298',
 '/opinion/columnistas/german-vargas-lleras/sabios-vs-populistas-columna-de-german-vargas-lleras-532238',
 '/opinion/columnistas/german-vargas-lleras',
 '/opinion/columnistas/german-vargas-lleras/sabios-vs-populistas-columna-de-german-vargas-lleras-532238',
 '/opinion/columnistas/german-vargas-lleras/sabios-vs-populistas-columna-de-german-vargas-lleras-532238',
 '/opinion/columnistas/maria-isabel-rueda/epm-patas-arriba-columna-de-maria-isabel-rueda-532288',
 '/opinion/columnistas/maria-isabel-rueda',
 '/opinion/columnistas/maria-isabel-rueda/epm-patas-arriba-col

In [167]:
r = requests.get(url)
r

<Response [200]>

In [168]:
if r.status_code == 200:
    pass
else:
    pass

In [169]:
bad_url = url.replace('e','i')
bad_url

'https://iltiimpo.com'

In [170]:
try:
    requests.get(bad_url)
except Exception as e:
    print('Error en la request \n')
    print(e)

Error en la request 

HTTPSConnectionPool(host='iltiimpo.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fdfde80bc50>: Failed to establish a new connection: [Errno -2] Name or service not known',))


### Descargando contenido

In [171]:
url_articulo_1 = articulos[0]
url + url_articulo_1

'https://eltiempo.com/opinion/editorial'

In [172]:
import datetime

def scrap_article(url_articulo):
    try:
        articulo = requests.get(url_articulo)
        if articulo.status_code == 200:
            s_articulo = BeautifulSoup(articulo.text,'lxml')
            # Extraer Título
            titulo = s_articulo.find('h1', attrs={'class':'title'})
            print(titulo.text)
            # Extraer volanta
            volanta = s_articulo.find('p', attrs={'class':'epigraph'})
            print(volanta.text)
            # Extraer Fecha
            fecha = s_articulo.find('span', attrs={'class':'fecha'})
            print(fecha.text)
            # Extraer Autor
            autor = s_articulo.find('span', attrs={'class':'nombre'})
            print(f'Por: {autor.text}')
            # Extraer texto
            textos = s_articulo.find_all('p', attrs={'class':'contenido'})
            for texto in textos:
                print(texto.text)
    except Exception as e:
        print('Error:')
        print(e)
        print('\n')

In [173]:
articulo_2 = str(url + articulos[1])
scrap_article(articulo_2)

Preso político
Álvaro Uribe es una nueva víctima de esta realidad que nos quieren imponer.

23 de agosto 2020 , 09:10 a.m.

Por: Tomás y Jerónimo Uribe Moreno
La majestad de la justicia es la consecuencia de la majestad de jueces, y ellos no son infalibles. La psicología conductual ha probado que los jueces no son ajenos a fallar instintivamente, a racionalizar en sus providencias sus afinidades o antipatías. Sufren, como todos, de sesgos cognitivos que nublan su racionalidad. Por lo anterior, tenemos el deber de cuestionar sus decisiones. Esto no puede ser descartado como amenaza institucional. No se puede cercenar el debate tildándolo de polarización. La polarización no la generan los discursos, sino los hechos, en este caso una detención preventiva, que para revestirla de aparente sofisticación necesitó mil quinientas páginas. Solo algo inexplicable requiere tantas páginas.
Pero esas son formas, nos preocupan los hechos. El proceso se ha filtrado sistemática y selectivamente a perio

In [191]:
a_3 = url + str(url + articulos[8])

scrap_article(a_3)

Error:
HTTPSConnectionPool(host='eltiempo.comhttps', port=443): Max retries exceeded with url: //eltiempo.com/opinion/columnistas/german-vargas-lleras/sabios-vs-populistas-columna-de-german-vargas-lleras-532238 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fdfde963dd8>: Failed to establish a new connection: [Errno -2] Name or service not known',))


